In [1]:
import pandas as pd

years = [23, 24, 25]

# unisci i dataframe di tutti gli anni in un unico dataframe con le dummies

df_final = pd.DataFrame()

for year in years:
    df = pd.read_csv(f"df_pre_dummies/df_{year}_con_pts_def_FINALE.csv")

    # drop_first=True to avoid collinearity during regression
    df = pd.get_dummies(df, columns=["EventType", "Surface"], prefix=["EventType", "Surface"], dtype=int, drop_first=True)

    df_final = pd.concat([df_final, df], ignore_index=True)

display(df_final)


,playercode,EventId,iscritto,Year,date_tournament,EventName,EventCountry,TotPrizeMoney,Same_Nationality,"IFNULL(`Rank`, 601)",ha_pts_def,EventType_500,Surface_Grass,Surface_Hard
0,mv14,891,1,2023,2023-01-02,Pune,India,642735,0,50,0,0,0,1
1,mv14,2843,0,2023,2023-01-02,Adelaide 1,Australia,642735,0,50,0,0,0,1
2,mw02,891,0,2023,2023-01-02,Pune,India,642735,0,51,0,0,0,1
3,mw02,2843,0,2023,2023-01-02,Adelaide 1,Australia,642735,0,51,0,0,0,1
4,mc65,891,0,2023,2023-01-02,Pune,India,642735,0,52,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53753,n0db,496,0,2025,2025-02-10,Marseille,France,740730,0,601,0,0,0,1
53754,n0db,499,0,2025,2025-02-10,Delray Beach,"FL, U.S.A.",680140,0,601,0,0,0,1
53755,n0db,424,0,2025,2025-02-03,Dallas,"TX, U.S.A.",2760000,0,601,0,1,0,1
53756,n0db,6932,0,2025,2025-02-17,Rio de Janeiro,Brazil,2396115,0,601,0,1,0,0


In [10]:
import statsmodels.api as sm
import numpy as np

df_final_copy = df_final.copy()

df_final_copy.rename(columns={"IFNULL(`Rank`, 601)": "Rank"}, inplace=True)

y = df_final_copy["iscritto"]

df_final_copy["TotPrizeMoney_log"] = np.log1p(df_final_copy["TotPrizeMoney"])
#df_final_copy["Rank_log"] = np.log1p(df_final_copy["Rank"])
X_fin = df_final_copy[["Rank", "TotPrizeMoney_log", "Same_Nationality", "ha_pts_def", "Surface_Grass", "Surface_Hard", "EventType_500"]]

# X_fin = df_final_copy.drop(columns=["iscritto", "playercode", "EventId", "Year", "date_tournament", "EventName", "EventCountry", "TotPrizeMoney"])

#normalizza le variabili con StandardScaler
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_fin = scaler.fit_transform(X)
# # rinomina le colonne con i nomi originali
# X_fin = pd.DataFrame(X_fin, columns=X.columns)

X_fin = sm.add_constant(X_fin)
model = sm.OLS(y, X_fin)
result = model.fit(clustered=True, cov_type='cluster', cov_kwds={'groups': df_final_copy['playercode']})

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               iscritto   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     258.7
Date:                Fri, 16 Jan 2026   Prob (F-statistic):          6.25e-159
Time:                        11:30:22   Log-Likelihood:                -7014.0
No. Observations:               53758   AIC:                         1.404e+04
Df Residuals:                   53750   BIC:                         1.412e+04
Df Model:                           7                                         
Covariance Type:              cluster                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.2607      0.10